In [55]:
import logging
logging.root.setLevel(logging.INFO)


In [56]:
import random

from xcs import XCSAlgorithm
from xcs.scenarios import Scenario

In [57]:
import random

from xcs.scenarios import Scenario
from xcs.bitstrings import BitString

class HaystackProblem(Scenario):
    
    def __init__(self, training_cycles=1000, input_size=50):
        self.input_size = input_size
        self.possible_actions = (True, False)
        self.initial_training_cycles = training_cycles
        self.remaining_cycles = training_cycles
        self.needle_index = random.randrange(input_size)
        self.needle_value = None

    def reset(self):
        self.remaining_cycles = self.initial_training_cycles
        haystack = BitString.random(self.input_size)
        self.needle_value = haystack[self.needle_index]
        
        sense_string = str(self.sense())
        raw_state = [str(s) for s in sense_string]
        return raw_state
        
    # XCS Hosford42 functions
    @property
    def is_dynamic(self):
        return False
        
    def get_possible_actions(self):
        return self.possible_actions
        
    def more(self):
        return self.remaining_cycles > 0
    
    def sense(self):
        haystack = BitString.random(self.input_size)
        self.needle_value = haystack[self.needle_index]
        return haystack
    
    def execute(self, action):
        self.remaining_cycles -= 1
        return action == self.needle_value

    # XCS Pyalcs functions
    def step(self, action):
        done = not self.execute(action)
        sense_string = str(self.sense())
        raw_state = [str(s) for s in sense_string]
        if not done:
            reward = 0
        else:
            reward = 1
        return raw_state, reward, done, _

In [58]:
logging.root.setLevel(logging.INFO)
scenario = HaystackProblem()

In [59]:
algorithm = XCSAlgorithm()
algorithm.exploration_probability = .1
algorithm.discount_factor = 0
algorithm.do_ga_subsumption = True
algorithm.do_action_set_subsumption = True

In [60]:
model = algorithm.new_model(scenario)
model.run(scenario, learn=True)


In [61]:
print(model)


00110##011###10##00###0#0##0001##11##1#0##1###01#1 => False
    Time Stamp: 750
    Average Reward: 0.0
    Error: 0.0
    Fitness: 8.5e-07
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
01#1#001###1#0#1#0##00##010#0###110100##01#1##0100 => True
    Time Stamp: 650
    Average Reward: 1.0
    Error: 0.0
    Fitness: 8.5e-07
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
01###001###1#0###0##00##010#01##1101000#01#1##0100 => True
    Time Stamp: 650
    Average Reward: 1.0
    Error: 0.0
    Fitness: 8.5e-07
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
1##0###1#1011101###1#0#0###1#0100#1111#0#100#00011 => False
    Time Stamp: 958
    Average Reward: 1.0
    Error: 0.0
    Fitness: 8.5e-07
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
0###00#1110001101111#011##00#111010100##0##10##0## => False
    Time Stamp: 53
    Average Reward: 0.0
    Error: 0.0
    Fitness: 0.15000850000000002
    Experience: 1
    Action Set Size: 1.0
    Numerosi

In [62]:
print(len(model))

200


In [63]:
for rule in model:
    if rule.fitness > .5 and rule.experience >= 10:
        print(rule.condition, '=>', rule.action, ' [%.5f]' % rule.fitness)

In [64]:
from lcs.agents.xcs import XCS, Configuration

cfg = Configuration(number_of_actions=4,
                    gamma=0,
                    metrics_trial_frequency=5,
                    covering_wildcard_chance=0.9
                    )


In [65]:
agent = XCS(cfg)
explore_population, explore_metrics = agent.explore(scenario, 1000, False)

In [66]:
for rule in explore_population:
    print(rule)


Cond:0001000011#1101001001101100011111100100#01010#0010 - Act:1 - Num:1 [fit: 0.000, exp: 0.00, pred: 0.000]
Cond:100011001011010110##1010001000111011#0111111001010 - Act:2 - Num:1 [fit: 0.000, exp: 0.00, pred: 0.000]
Cond:10#0#1#100001#001010#1#00010011101#0#1000100100110 - Act:1 - Num:1 [fit: 0.050, exp: 1.00, pred: 0.388]
Cond:0##0111111001111100000#100101110000010001010#10101 - Act:2 - Num:1 [fit: 0.000, exp: 0.00, pred: 0.000]
Cond:0110010111111010001110001100001#1#0011010101001011 - Act:2 - Num:1 [fit: 0.000, exp: 0.00, pred: 0.000]
Cond:##01#1101#101000000010111#00111001010010101#011010 - Act:1 - Num:1 [fit: 0.000, exp: 0.00, pred: 0.000]
Cond:#11001101010011001010011010#00001#1#011101000000#1 - Act:1 - Num:1 [fit: 0.000, exp: 0.00, pred: 0.000]
Cond:0011#10#010#0#00010100001100000100111#100110010100 - Act:0 - Num:1 [fit: 0.000, exp: 0.00, pred: 0.000]
Cond:010#1#10010010101110110#10011100100#01010110111101 - Act:3 - Num:1 [fit: 0.050, exp: 1.00, pred: 1.000]
Cond:#10100#000#000

In [67]:
print(len(explore_population))

200


In [68]:
for rule in explore_population:
    if rule.fitness > .5 and rule.experience >= 10:
        print(rule.condition, '=>', rule.action, ' [%.5f]' % rule.fitness)